<a href="https://colab.research.google.com/github/JaswanthMannem/CREWAI/blob/main/crewai_3_tools_customer_outreach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain-openai

In [ ]:
!pip install langchain_huggingface

In [37]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [38]:
import os

os.makedirs('./instructions', exist_ok=True)

from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    os.rename(filename, './instructions/' + filename)

Saving instruction.txt to instruction.txt


In [39]:
from google.colab import userdata
serp_key=userdata.get('SERP_API_KEY')
openai_key=userdata.get('OPENAI_API_KEY')

In [40]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

In [54]:
import os

openai_api_key = openai_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serp_key

In [42]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(openai_api_key=openai_api_key),
)

In [43]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=ChatOpenAI(openai_api_key=openai_api_key),
)

In [44]:
from crewai_tools import DirectoryReadTool, FileReadTool, SerperDevTool

In [45]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

In [46]:
from crewai_tools import BaseTool

In [47]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")

    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [48]:
sentiment_analysis_tool = SentimentAnalysisTool()

In [49]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

In [50]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

In [51]:
crew = Crew(
    agents=[sales_rep_agent,
            lead_sales_rep_agent],

    tasks=[lead_profiling_task,
           personalized_outreach_task],
    embedder={
            "provider": "huggingface",
            "config": {"model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"},
        },
    verbose=2,
    memory=True
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [52]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
I need to gather as much information as possible about DeepLearningAI to create a comprehensive report for our sales team.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company overview"} 


Search results: Title: Deeplearning.ai - Crunchbase Company Profile & Funding
Link: https://www.crunchbase.com/organization/deeplearning-ai
Snippet: Deeplearning.ai is company that explor

In [53]:
from IPython.display import Markdown
Markdown(result)

Email Draft for Andrew Ng, CEO of DeepLearningAI:

Subject: Exciting Collaboration Opportunity with CrewAI for DeepLearningAI's Recent Product Launch

Dear Mr. Ng,

I hope this email finds you well. I recently came across the exciting news of DeepLearningAI's recent product launch and wanted to reach out to discuss a potential collaboration opportunity with CrewAI.

As a leader in the AI education sector, DeepLearningAI has continuously impressed the industry with its commitment to making AI education accessible to a global audience. With over 7 million learners, your platform has become a beacon of knowledge and skill-building in the AI field.

Our team at CrewAI is equally passionate about empowering organizations like DeepLearningAI to achieve their goals through personalized engagement strategies. We believe that our solutions can enhance your existing educational offerings and help identify high-value leads that align with your mission of building an AI-powered future.

The recent launch of your AI mentoring product, featuring an AI avatar of yourself, showcases your dedication to innovation and excellence. We see great potential in collaborating with DeepLearningAI to further elevate this initiative and provide valuable insights that drive mutual growth in the AI industry.

I would love the opportunity to discuss how CrewAI can support DeepLearningAI's goals and contribute to your continued success. Please let me know a convenient time for a meeting or call to explore this exciting collaboration further.

Thank you for considering this partnership opportunity, and I look forward to the possibility of working together to shape the future of AI education.

Warm regards,

[Your Name]
Lead Sales Representative
CrewAI

Email Draft 2:

Subject: Elevating DeepLearningAI's Educational Initiatives with CrewAI's Strategic Solutions

Dear Mr. Ng,

I hope this email finds you well. I am reaching out from CrewAI with a shared vision of empowering the global workforce through high-quality AI education, a vision that aligns closely with DeepLearningAI's mission.

Your recent collaboration with Coursera on the "Generative AI for Everyone" course exemplifies your commitment to providing accessible AI training to individuals worldwide. We believe that our strategic solutions at CrewAI can further enhance your educational initiatives and drive greater engagement among your learners.

By leveraging our cutting-edge tools and expertise in personalized engagement strategies, we can help DeepLearningAI identify and nurture leads that resonate with your company values. Our tailored approach is designed to support your goal of building an AI-powered future through education, ensuring that your platform continues to inspire and educate millions of learners globally.

I am excited about the prospect of collaborating with DeepLearningAI to elevate your educational offerings and contribute to the growth of the AI industry as a whole. Please let me know if you would be interested in exploring this partnership further, and I would be more than happy to discuss how CrewAI can support your goals.

Thank you for considering this opportunity, and I look forward to the possibility of working together to drive innovation and excellence in AI education.

Best regards,

[Your Name]
Lead Sales Representative
CrewAI